In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install 
from pypfopt import black_litterman
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import plotting
from scipy.optimize import minimize
!pip install cvxpy
import cvxpy as cp

In [2]:
def minimum_variance(ret):
    def find_port_variance(weights):
        cov = ret.cov()
        port_var = np.sqrt(np.dot(weights.T, np.dot(cov, weights)))
        return port_var

    def weight_cons(weights):
        return np.sum(weights) - 1



    bounds_lim = [(0, 1) for x in range(len(ret.columns))] # change to (-1, 1) if you want to short
    init = [1/len(ret.columns) for i in range(len(ret.columns))]
    constraint = {'type': 'eq', 'fun': weight_cons}

    optimal = minimize(fun=find_port_variance,
                       x0=init,
                       bounds=bounds_lim,
                       constraints=constraint,
                       method='SLSQP'
                       )

    return list(optimal['x'])

In [3]:
data = pd.read_csv('correct_data.csv')
data = data.set_index('Date')
returns = data[['BTC', 'LTC', 'XRP', 'DASH']].pct_change().dropna()
rf = data['RF'].shift().dropna()

In [4]:
def bl_vbc(returns, rf, window, c, lamb, dirach, P):

  sharpe_tc = []
  sharpe_tc1 = []
  port_std = []
  tc = 50*10^-4
  tc_1 = 10^-3


  n = len(returns.iloc[0])

  for i in range(len(returns.iloc[window:])):

    x_bench = minimum_variance(returns.iloc[i:i+window])
    Q = returns.iloc[i:i+window].mean()*52
    cov = returns.iloc[i:i+window].cov()*52
    omega = 1/dirach * P @ cov @ P.T
    H = lamb * cov @ x_bench
    bl_mean = np.linalg.inv(np.linalg.inv((c*cov)) + P.T @ np.linalg.inv(omega) @ P) @ (np.linalg.inv(c*cov)@H + P.T @ np.linalg.inv(omega) @ Q.T)
    bl_sigma = cov + np.linalg.inv(np.linalg.inv(c*cov) + P.T @ np.linalg.inv(omega) @ P)

    ef = EfficientFrontier(bl_mean, bl_sigma, solver=None)

    ef.add_constraint(lambda x: cp.abs(x[0]-1/n)*returns.iloc[i:i+window,0].std()/returns.iloc[i:i+window].std().mean()<=.2)
    ef.add_constraint(lambda x: cp.abs(x[1]-1/n)*returns.iloc[i:i+window,1].std()/returns.iloc[i:i+window].std().mean()<=.2)
    ef.add_constraint(lambda x: cp.abs(x[2]-1/n)*returns.iloc[i:i+window,2].std()/returns.iloc[i:i+window].std().mean()<=.2)
    ef.add_constraint(lambda x: cp.abs(x[3]-1/n)*returns.iloc[i:i+window,3].std()/returns.iloc[i:i+window].std().mean()<=.2)

    covar = ef.cov_matrix
    weights = list(ef.max_sharpe(risk_free_rate=rf.iloc[i:i+window].mean()).values())
    std = np.sqrt(np.dot(np.dot(covar, weights), weights))
    sharpe_tc.append((ef.expected_returns - rf.iloc[i:i+window].mean()-tc)/std)
    sharpe_tc1.append((ef.expected_returns - rf.iloc[i:i+window].mean()-tc)/std)
    port_std.append(std)

  output = pd.DataFrame(np.array([sharpe_tc, sharpe_tc1, port_std]).T, columns=['Sharpe Ratio (50bps TC)', 'Sharpe Ratio (100bps TC)', 'Portfolio Standard Deviation'])
  output = output.set_index(returns.iloc[window:].index)
  return output


In [ ]:
def bl_vbc_longonly(returns, rf, window, c, lamb, dirach, P):
  sharpe_tc = []
  sharpe_tc1 = []
  port_std = []


  n = len(returns.iloc[0])

  for i in range(len(returns.iloc[window:])):

    x_bench = minimum_variance(returns.iloc[i:i+window])
    Q = returns.iloc[i:i+window].mean()*52
    cov = returns.iloc[i:i+window].cov()*52
    omega = 1/dirach * P @ cov @ P.T
    H = lamb * cov @ x_bench
    bl_mean = np.linalg.inv(np.linalg.inv((c*cov)) + P.T @ np.linalg.inv(omega) @ P) @ (np.linalg.inv(c*cov)@H + P.T @ np.linalg.inv(omega) @ Q.T)
    bl_sigma = cov + np.linalg.inv(np.linalg.inv(c*cov) + P.T @ np.linalg.inv(omega) @ P)

    ef = EfficientFrontier(bl_mean, bl_sigma, solver=None)

    ef.add_constraint(lambda x: cp.abs(x[0]-1/n)*returns.iloc[i:i+window,0].std()/returns.iloc[i:i+window].std().mean()<=.2)
    ef.add_constraint(lambda x: cp.abs(x[1]-1/n)*returns.iloc[i:i+window,1].std()/returns.iloc[i:i+window].std().mean()<=.2)
    ef.add_constraint(lambda x: cp.abs(x[2]-1/n)*returns.iloc[i:i+window,2].std()/returns.iloc[i:i+window].std().mean()<=.2)
    ef.add_constraint(lambda x: cp.abs(x[3]-1/n)*returns.iloc[i:i+window,3].std()/returns.iloc[i:i+window].std().mean()<=.2)
    ef.add_constraint(lambda x: x[0] >= 0)
    ef.add_constraint(lambda x: x[1] >= 0)
    ef.add_constraint(lambda x: x[2] >= 0)
    ef.add_constraint(lambda x: x[3] >= 0)

    covar = ef.cov_matrix
    weights = list(ef.max_sharpe(risk_free_rate=rf.iloc[i:i+window].mean()).values())
    std = np.sqrt(np.dot(np.dot(covar, weights), weights))
    sharpe_tc.append((ef.expected_returns - rf.iloc[i:i+window].mean()-tc)/std)
    sharpe_tc1.append((ef.expected_returns - rf.iloc[i:i+window].mean()-tc)/std)
    port_std.append(std)

  output = pd.DataFrame(np.array([sharpe_tc, sharpe_tc1, port_std]).T, columns=['Sharpe Ratio (50bps TC)', 'Sharpe Ratio (100bps TC)', 'Portfolio Standard Deviation'])
  output = output.set_index(returns.iloc[window:].index)
  return output


In [ ]:
c = 0.1625
P = np.array(
         [[1, 0, 0, 0],
          [0, 1, 0, 0],
          [0, 0, 1, 0],
          [0, 0, 0, 1]]
  )
lamb = 1
dirach = 1
window = 110

bl_vbc(returns, rf, window, c, lamb, dirach, P)

,Sharpe Ratio,Portfolio Standard Deviation
Date,,
2016/4/8,0.475178,0.706164
2016/4/15,0.488818,0.709920
2016/4/22,0.464832,0.693058
2016/4/29,0.472627,0.697091
2016/5/6,0.481617,0.688810
...,...,...
2018/4/6,0.955485,0.852394
2018/4/13,0.946518,0.855566
2018/4/20,0.963094,0.864489
